In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_pinball_loss
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
import joblib

In [ ]:
df = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/HEFTcom24/data/features.csv")

In [ ]:
df = df.drop(columns=['Unnamed: 0'])   
df = df.dropna()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=["valid_time", "Solar_MWh_credit"]), df["Solar_MWh_credit"], test_size=0.2, random_state=42, shuffle= False)

In [ ]:
import numpy as np
import pymc3 as pm
import theano.tensor as tt
from sklearn.model_selection import train_test_split
import pandas as pd

# Beispiel-Daten laden
df = pd.DataFrame({
    'valid_time': pd.date_range('2023-01-01', periods=100, freq='H'),
    'feature_1': np.random.randn(100),
    'feature_2': np.random.randn(100),
    'Solar_MWh_credit': np.random.randn(100)
})

# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=["valid_time", "Solar_MWh_credit"]),
    df["Solar_MWh_credit"],
    test_size=0.2, 
    random_state=42, 
    shuffle=False
)

# Formate für PyMC3 vorbereiten
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

n_train = len(y_train)
n_test = len(y_test)

# Bayesianisches State-Space-Modell trainieren
with pm.Model() as model:
    
    # Latenter Zustand x_t (hier nehmen wir an, dass feature_1 der latente Zustand ist)
    sigma_w = pm.HalfNormal('sigma_w', 1.0)  # Prozessrauschen
    A = pm.Normal('A', 0, 1)  # Übergangsmatrix
    
    x_0 = pm.Normal('x_0', 0, 1)  # Initialer Zustand
    x = [x_0]  # Liste zur Speicherung der Zustände
    
    # Latente Zustände für Trainingsdaten
    for t in range(1, n_train):
        x_t = pm.Normal(f'x_{t}', mu=A * x[t-1], sigma=sigma_w)
        x.append(x_t)
    
    # In Theano Tensor konvertieren
    x = tt.stack(x)
    
    # Beobachtungsmodell: Hier Laplace für Quantilregression, aber du kannst auch Normal wählen
    sigma_v = pm.HalfNormal('sigma_v', 1.0)  # Beobachtungsrauschen
    
    # Beobachtungen basierend auf latenten Zuständen und Prädiktoren
    mu_y = x + pm.math.dot(X_train, pm.Normal('beta', mu=0, sigma=1, shape=X_train.shape[1]))  # lineares Modell
    y_obs = pm.Laplace('y_obs', mu=mu_y, b=sigma_v, observed=y_train)

    # Sample posterior
    trace = pm.sample(1000, tune=2000, return_inferencedata=True)

# Vorhersagen auf dem Test-Set machen
with model:
    # Samplen des posterior predictives für das Test-Set
    x_last_train = np.mean([trace.posterior[f'x_{n_train-1}'].values])  # letzter Zustand aus dem Training
    
    predictions = []
    
    for t in range(n_test):
        # Vorhersage des nächsten latenten Zustands basierend auf dem letzten Zustand des Trainingsdatensatzes
        A_samples = trace.posterior['A'].values
        sigma_w_samples = trace.posterior['sigma_w'].values
        x_future = A_samples * x_last_train + np.random.normal(0, sigma_w_samples)
        x_last_train = np.mean(x_future)  # Update des letzten Zustands
        
        # Vorhersage der Beobachtung für das Test-Set (Quantilregression)
        sigma_v_samples = trace.posterior['sigma_v'].values
        y_future = x_last_train + np.dot(X_test[t], trace.posterior['beta'].values) + np.random.laplace(0, sigma_v_samples)
        predictions.append(np.mean(y_future))

# Ergebnisse anzeigen
print("Vorhersagen für das Testset:", predictions)